In [148]:
import pandas as pd

In [149]:
df=pd.read_csv('spotify_artist_data.csv')



In [150]:
df.rename(columns={"Artist Name": 'Artist_Name', "Lead Streams": 'Lead_Streams'}, inplace=True)
df['Lead_Streams'] = df['Lead_Streams'].str.replace(',', '')
df['Feats'] = df['Feats'].str.replace(',', '') 



In [151]:
df

,Unnamed: 0,Artist_Name,Lead_Streams,Feats,Tracks,One Billion,100 Million,Last Updated
0,1,Drake,50162292808,19246513666,262,6,130,19.09.22
1,2,Bad Bunny,44369032140,5391990975,163,5,118,20.09.22
2,3,Ed Sheeran,38153682361,2791278201,240,10,62,19.09.22
3,4,The Weeknd,34767779741,4288903657,186,8,72,19.09.22
4,5,Taylor Swift,32596728109,424053296,323,1,96,20.09.22
...,...,...,...,...,...,...,...,...
995,1005,Gotye,1424591618,10576378,46,1,1,31.07.22
996,1006,NIKI,1422920850,231405433,55,0,4,14.09.22
997,1007,Duke Dumont,1422096182,24622115,121,0,2,31.07.22
998,1008,Laura Pausini,1421622604,293291443,434,0,0,13.09.22


In [152]:
df.set_index("Artist_Name", inplace=True, drop=True)

In [153]:
df['Lead_Streams'] = pd.to_numeric(df['Lead_Streams'])
df['Feats'] = pd.to_numeric(df['Feats'])


In [154]:
df.drop(['Unnamed: 0', 'One Billion', "100 Million", 'Last Updated', 'Tracks'], axis=1, inplace=True)

In [155]:
df

,Lead_Streams,Feats
Artist_Name,,
Drake,50162292808,19246513666
Bad Bunny,44369032140,5391990975
Ed Sheeran,38153682361,2791278201
The Weeknd,34767779741,4288903657
Taylor Swift,32596728109,424053296
...,...,...
Gotye,1424591618,10576378
NIKI,1422920850,231405433
Duke Dumont,1422096182,24622115


In [156]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, Drake to Beck
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Lead_Streams  1000 non-null   int64
 1   Feats         1000 non-null   int64
dtypes: int64(2)
memory usage: 23.4+ KB


In [157]:
df_norm = df.divide(df.sum(axis=1), axis=0)
df_norm.head(10)

,Lead_Streams,Feats
Artist_Name,,
Drake,0.722708,0.277292
Bad Bunny,0.891642,0.108358
Ed Sheeran,0.931829,0.068171
The Weeknd,0.890188,0.109812
Taylor Swift,0.987158,0.012842
Justin Bieber,0.750100,0.249900
Ariana Grande,0.940236,0.059764
Eminem,0.879613,0.120387
Post Malone,0.916216,0.083784


In [158]:
df_norm.sum(axis=1).head(20)

Artist_Name
Drake              1.0
Bad Bunny          1.0
Ed Sheeran         1.0
The Weeknd         1.0
Taylor Swift       1.0
Justin Bieber      1.0
Ariana Grande      1.0
Eminem             1.0
Post Malone        1.0
BTS                1.0
J Balvin           1.0
Kanye West         1.0
Billie Eilish      1.0
Juice WRLD         1.0
Coldplay           1.0
Dua Lipa           1.0
Imagine Dragons    1.0
XXXTENTACION       1.0
Ozuna              1.0
Khalid             1.0
dtype: float64

In [159]:
from scipy.sparse import lil_matrix # Needed for building the matrix of user ratings
import scipy.spatial.distance # Needed for calculating pairwise distances

In [160]:
artist_name_map = {index: "" for index in df.index}

In [161]:
artist_name_map

{'Drake': '',
 'Bad Bunny': '',
 'Ed Sheeran': '',
 'The Weeknd': '',
 'Taylor Swift': '',
 'Justin Bieber': '',
 'Ariana Grande': '',
 'Eminem': '',
 'Post Malone': '',
 'BTS': '',
 'J Balvin': '',
 'Kanye West': '',
 'Billie Eilish': '',
 'Juice WRLD': '',
 'Coldplay': '',
 'Dua Lipa': '',
 'Imagine Dragons': '',
 'XXXTENTACION': '',
 'Ozuna': '',
 'Khalid': '',
 'Rihanna': '',
 'Travis Scott': '',
 'Maroon 5': '',
 'David Guetta': '',
 'Shawn Mendes': '',
 'Bruno Mars': '',
 'Daddy Yankee': '',
 'Calvin Harris': '',
 'Kendrick Lamar': '',
 'Sam Smith': '',
 'Queen': '',
 'Beyoncé': '',
 'Adele': '',
 'Future': '',
 'One Direction': '',
 'The Chainsmokers': '',
 'Chris Brown': '',
 'Lady Gaga': '',
 'J. Cole': '',
 'Nicki Minaj': '',
 'Anuel AA': '',
 'Halsey': '',
 'The Beatles': '',
 'Selena Gomez': '',
 'Sia': '',
 'Maluma': '',
 'Harry Styles': '',
 'Marshmello': '',
 'Lil Uzi Vert': '',
 'Linkin Park': '',
 'Twenty One Pilots': '',
 'Rauw Alejandro': '',
 'Katy Perry': '',
 'Kyg

In [162]:
target_artist='ILLENIUM'
target_artist1='SZA'
target_artist2='Lewis Capaldi'

In [163]:

target_artist_stats = df_norm.loc[target_artist]
distances = scipy.spatial.distance.cdist(df_norm, [target_artist_stats], metric="euclidean")[:,0]
query_distances = list(zip(df.index, distances))

for similar_artist, similar_stats in sorted(query_distances, key=lambda x: x[1], reverse=False)[:10]:
     print(similar_artist, artist_name_map[similar_artist], similar_stats, df.loc[similar_artist].sum())

ILLENIUM  0.0 7982319226
Lil Wayne  0.003392423621751566 23130373255
DJ Luian  0.0058555373069129136 4486557401
Ty Dolla $ign  0.008796250161742238 17151496921
Tainy  0.010717006966969587 8615920146
Wesley Safadão  0.011889337237975176 9390980795
2 Chainz  0.024804338706035278 8511545529
Snoop Dogg  0.0345145265271461 10493061972
Soda Stereo  0.03532604780044659 7310719654
Mambo Kingz  0.03635730854294034 4244678260


In [164]:
target_artist_stats = df_norm.loc[target_artist1]
distances = scipy.spatial.distance.cdist(df_norm, [target_artist_stats], metric="euclidean")[:,0]
query_distances = list(zip(df.index, distances))

for similar_artist, similar_stats in sorted(query_distances, key=lambda x: x[1], reverse=False)[:10]:
     print(similar_artist, artist_name_map[similar_artist], similar_stats, df.loc[similar_artist].sum())

SZA  0.0 9417167939
Tove Lo  0.0013920493250995836 5989538400
Saweetie  0.0019973177431830253 2595842076
Frank Ocean  0.0026593052585039633 10184261128
Alicia Keys  0.002844060987293976 6205592801
Playboi Carti  0.003121441091145964 7435726003
Bryson Tiller  0.0031455326315153525 5626455814
Justin Timberlake  0.0037388199018465046 9392500063
Los Ángeles Azules  0.004566193250827201 3897312543
Calibre 50  0.0050834302165100684 4977050050


In [165]:
target_artist_stats = df_norm.loc[target_artist2]
distances = scipy.spatial.distance.cdist(df_norm, [target_artist_stats], metric="euclidean")[:,0]
query_distances = list(zip(df.index, distances))

for similar_artist, similar_stats in sorted(query_distances, key=lambda x: x[1], reverse=False)[:10]:
     print(similar_artist, artist_name_map[similar_artist], similar_stats, df.loc[similar_artist].sum())

Lewis Capaldi  0.0 6958007662
Oasis  4.413684270813763e-08 4924852716
Jason Aldean  1.4077269068736914e-07 3797817377
NLE Choppa  2.4976457451251036e-07 3018675405
Billie Eilish  2.815009524398165e-07 23684986780
Coldplay  2.815009524398165e-07 22214835214
One Direction  2.815009524398165e-07 15499795436
The Beatles  2.815009524398165e-07 14001404418
Harry Styles  2.815009524398165e-07 13347420670
Twenty One Pilots  2.815009524398165e-07 12949146668
